In [1]:
import pandas as pd
from datasets import load_from_disk
from nltk.corpus import stopwords
import joblib
import pandas as pd
from sklearn.metrics import roc_auc_score
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve,confusion_matrix
import plotly.graph_objects as go
import plotly.figure_factory as ff
from transformers import pipeline,AutoTokenizer,AutoModelForSequenceClassification

In [2]:

pipeline_nb = joblib.load("./models/nb_classifier_pipeline.joblib")


label2id = {'Nohate':0, 'Hate':1}
id2label = {v: k for k, v in label2id.items()}
tokenizer = AutoTokenizer.from_pretrained('chgrdj/nlp-assignment-Bert')
model = AutoModelForSequenceClassification.from_pretrained('chgrdj/nlp-assignment-Bert', 
                                            label2id=label2id, 
                                            id2label=id2label,
                                            num_labels=len(label2id))
pipeline_tfm = pipeline('text-classification', model=model,tokenizer=tokenizer)

/home/cgrdj/Documents/code/challenges/nlp_assignment-master/nlp-assignment-env/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [3]:

test_ds=load_from_disk('data/test_ds')
test_texts=test_ds['text']
test_labels=test_ds['label']

In [4]:
# Predict on the test set
y_pred_nb = pipeline_nb.predict(test_texts)
y_prob_nb = pipeline_nb.predict_proba(test_texts)[:, 1]  # Probabilities for the positive class

# Compute metrics
nb_dict={"accuracy" : accuracy_score(test_labels, y_pred_nb),
"precision" : precision_score(test_labels, y_pred_nb),
"recall" : recall_score(test_labels, y_pred_nb),
"f1" : f1_score(test_labels, y_pred_nb),
"roc_auc" : roc_auc_score(test_labels, y_prob_nb)}

In [5]:
res=pipeline_tfm(test_texts)
y_pred_tfm=[]
y_prob_tfm=[]
for d in res:
    y_pred_tfm.append(label2id[d['label']])
    if label2id[d['label']]:
        y_prob_tfm.append(d['score']) 
    else : 
        y_prob_tfm.append(1-d['score'])

tfm_dict={"accuracy" : accuracy_score(test_labels, y_pred_tfm),
"precision" : precision_score(test_labels, y_pred_tfm),
"recall" : recall_score(test_labels, y_pred_tfm),
"f1" : f1_score(test_labels, y_pred_tfm),
"roc_auc" : roc_auc_score(test_labels, y_prob_tfm)}

In [6]:
precision_score(test_labels, y_pred_tfm)

0.7720588235294118

In [7]:

df_metrics = pd.DataFrame([nb_dict, tfm_dict],index=['Naive Bayes','BERT'])
df_metrics.to_csv('./results/metrics.csv')
df_metrics

,accuracy,precision,recall,f1,roc_auc
Naive Bayes,0.730126,0.713178,0.769874,0.740443,0.766566
BERT,0.809623,0.772059,0.878661,0.821918,0.900212


In [8]:

conf_matrix_nb = confusion_matrix(test_labels, y_pred_nb)
conf_matrix_tfm = confusion_matrix(test_labels, y_pred_tfm)

In [9]:
labels = ['Negative', 'Positive']

fig_cm_nb = ff.create_annotated_heatmap(z=conf_matrix_nb, x=labels, y=labels, colorscale='Greens')
fig_cm_nb.update_layout(title='Confusion Matrix Naive bayes',
                      xaxis_title='Predicted Label',
                      yaxis_title='True Label')
fig_cm_nb.show()
fig_cm_nb.write_image("./results/confusion_matrix_nb.png")


fig_cm_tfm = ff.create_annotated_heatmap(z=conf_matrix_tfm, x=labels, y=labels, colorscale='Blues')
fig_cm_tfm.update_layout(title='Confusion Matrix TFM',
                      xaxis_title='Predicted Label',
                      yaxis_title='True Label')
fig_cm_tfm.show()
fig_cm_tfm.write_image("./results/confusion_matrix_tfm.png")

In [10]:
fpr_nb, tpr_nb, _ = roc_curve(test_labels, y_prob_nb)
fpr_tfm, tpr_tfm, _ = roc_curve(test_labels, y_prob_tfm)

fig = go.Figure()

fig.add_trace(go.Scatter(x=fpr_nb, y=tpr_nb, mode='lines', name='NB ROC curve (area = %0.2f)' % nb_dict['roc_auc']))
fig.add_trace(go.Scatter(x=fpr_tfm, y=tpr_tfm, mode='lines', name='TFM ROC curve (area = %0.2f)' % tfm_dict['roc_auc']))
fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', line=dict(dash='dash'), name='Random'))

fig.update_layout(
    title='ROC Curve',
    xaxis_title='FPR',
    yaxis_title='TPR',
    showlegend=True
)

fig.show()
fig_cm_tfm.write_image("./results/Roc_curve.png")